# Prediciting Post Reaction using Naive Bayes

## Import Libraries

### Main Libraries

In [1]:
import pandas as pd
import numpy as np
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Scikit Learn

In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Load Dataset

In [3]:
df_train = pd.read_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/results/postSenti_train.tsv', index_col='Unnamed: 0', sep='\t')
df_test = pd.read_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/results/postSenti_test.tsv', index_col='Unnamed: 0', sep='\t') 

In [4]:
# Separate features and target for training dataset
X_train = df_train.drop(['sentiment_class'], axis=1)
y_train = df_train['sentiment_class']

In [5]:
# Separate features and target for testing dataset
X_test = df_test.drop(['sentiment_class'], axis=1)
y_test = df_test['sentiment_class']

## Naive Bayes Classification

### Feature selection using RandomForestClassifier--feature_importances

In [6]:
# Instantiate RandomForestClassifier
rf = RandomForestClassifier(random_state=35).fit(X_train, y_train)
rf_feat_imp = rf.feature_importances_
# Feature importance
feature_importance = pd.DataFrame(rf_feat_imp, index=X_train.columns, columns=["Feat_Imp"])

In [7]:
#Sort in descending order to get Most Important to Least Important
feature_importance.sort_values("Feat_Imp",ascending=False, inplace=True)
feature_importance.reset_index(inplace=True)

In [8]:
# Top 5 Feature Importance
top5_feature = feature_importance.iloc[:5]
predictor = list(top5_feature['index'])
predictor

['Topic 0', 'hour_day', 'Topic 1', 'Topic 2', 'Topic 7']

### Hyperparameter Optimization using GridSearch

In [9]:
param_grid = {'alpha': [1, 0.5, 0.1, 0.05, 0.01, 0.005],
              'class_prior': [None],
              'fit_prior': [True]}

In [10]:
# Instantiate NB
clf = MultinomialNB()
# Instantiate GridSearchCV
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)

In [11]:
# Fit X_train; Squared the train dataset because the model does not accept negative values
grid_search.fit(X_train[predictor]**2, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [1, 0.5, 0.1, 0.05, 0.01, 0.005], 'class_prior': [None], 'fit_prior': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
print("Best estimator:")
print(grid_search.best_estimator_)
print("Best parameters:")
print(grid_search.best_params_)
print("Best score:")
print(grid_search.best_score_)

Best estimator:
MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
Best parameters:
{'alpha': 1, 'class_prior': None, 'fit_prior': True}
Best score:
0.5480153649167734


### Build NB Model

In [13]:
# Instantiate NB
NB = MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
# Fit the training dataset on the NB; Squared the test dataset because the model does not accept negative values
NB.fit(X_train[predictor]**2, y_train)
# Predict the labels on validation dataset
y_pred = NB.predict(X_test[predictor]**2)

### Evaluate NB Model

In [14]:
# Get Metrics
accuracy = accuracy_score(y_pred, y_test)*100
# Store metrics in dictionary
score_dict = {'NaiveBayes': pd.Series([accuracy], index=['AccuracyScore'])}
# Transform dictionary to df
df_metrics = pd.DataFrame(score_dict)
df_metrics

,NaiveBayes
AccuracyScore,57.653061


## Cross Validation

In [15]:
print('Mean Cross validated score -> ', np.mean(cross_val_score(NB, X_train[predictor]**2, y_train, cv=5)*100))

Mean Cross validated score ->  54.80495440315775
